In [0]:
# 확률 분포 가중치 값 바꾸기
import numpy as np

def reweight_distribution(original_distribution, temperature = 0.5):
  distribution = np.log(original_distribution) / temperature
  distribution = np.exp(distribution)
  return distribution / np.sum(distribution)

  # 모든 수에 log하고 exp해주면 똑같아지는데 log해서 값 잠깐 줄이고 temperature로 나눠서
  # 다시 exp해줌.

In [12]:
reweight_distribution([0.4, 0.6])

array([0.30769231, 0.69230769])

In [44]:
reweight_distribution([0.4, 0.6], temperature = 1)

array([0.4, 0.6])

In [15]:
reweight_distribution([0.1, 0.9], temperature = 0.6)
# 온도가 높을수록 엔트로피를 더 늘려줌

array([0.02503709, 0.97496291])

In [19]:
reweight_distribution([0.4, 0.6], temperature = 0.2)
# 온도가 낮을수록 엔트로피를 더 줄여줌(탐욕적)

array([0.11636364, 0.88363636])

# 글자 수준의 LSTM 텍스트 생성 모델 구현

In [40]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('말뭉치 크기: ', len(text))

Using TensorFlow backend.


606208/600901 [==============================] - 0s 0us/step
말뭉치 크기:  600893


In [41]:
type(text)

str

In [43]:
# maxlen 길이를 가진 시퀀스를 중복하여 추출
# 추출된 시퀀스를 원-핫 인코딩으로 변환하고 
# shape(sequences, maxlen, unique_characers)로 합침

# 60개 글자로 된 시퀀스 추출
maxlen = 60

# 세 글자씩 띄면서 시퀀스 샘플링할 거임 tri-gram
step = 3

# 추출할 시퀀스를 담을 리스트
sentences = []

# 시퀀스 다음 글자를 담을 리스트
next_chars = []

for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])
print('시퀀스 개수: ', len(sentences))

# 말뭉치에서 고유한 글자를 담을 리스트
chars = sorted(list(set(text))) #unique
print('고유한 글자: ', len(chars))

# chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리
char_indices = dict((char, chars.index(char)) for char in chars)

# 글자를 one hot 해서 0과 1의 이진 배열로 바꿈
x = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)

for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

시퀀스 개수:  200278
고유한 글자:  57


## 네트워크 구성
- 훈련된 모델과 시드로 쓰일 텍스트가 주어지면 아래처럼 반복해서 텍스트 생성할 수 있음
1. 지금까지 만든 걸로 모델에 주입해서 다음 글자에 대한 확률 분포 뽑음
2. 특정 온도로 확률 가중치 조절
3. 가중치 조정된 분포에서 새로운 글자 샘플링
4. 새로운 글자를 생성된 텍스트 끝에 추가

In [51]:
# LSTM 뒤에 Dense 붙음
# 가능한 모든 글자에 softmax 출력 붙음
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape = (maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation = 'softmax'))

In [54]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [0]:
def sample(preds, temperature = 1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [0]:
# 반복적으로 훈련하고 텍스트 생성.
# 1epoch마다 학습 후 여러가지 온도를 사용해서 텍스트를 생성함.
# 이러면 진화과정을 볼 수 있고 온도가 샘플링 전략에 미치는 영향도 보여줌.

In [57]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

# 60 에포크 동안 모델을 훈련합니다
for epoch in range(1, 60):
    print('에포크', epoch)
    # 데이터에서 한 번만 반복해서 모델을 학습합니다
    model.fit(x, y, batch_size=128, epochs=1)

    # 무작위로 시드 텍스트를 선택합니다
    seed_text = text[start_index: start_index + maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')

    # 여러가지 샘플링 온도를 시도합니다
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)

        # 시드 텍스트에서 시작해서 400개의 글자를 생성합니다
        for i in range(400):
            # 지금까지 생성된 글자를 원-핫 인코딩으로 바꿉니다
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            # 다음 글자를 샘플링합니다
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
Epoch 1/1
 46208/200278 [=====>........................] - ETA: 2:27 - loss: 1.3780

KeyboardInterrupt: ignored

In [56]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored